<a href="https://colab.research.google.com/github/ming-90/Kaggle_Study/blob/main/5.%20TrashClassifier/Trash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [이미지/이지AI/이미지 분류] 수중 해양 쓰레기 종류 분류 모델

이 노트북은 셀을 차례로 실행하여 수치 분류 과제의 전반적인 과정을 수행해볼 수 있게 제작되었습니다. 파일 경로를 추가하는 것 외에는 큰 코드 수정 없이 실습해 볼 수 있습니다.

## 과제 설명
해양 침적 쓰레기 이미지로부터 이미지 내에 존재하는 해양 쓰레기 유형을 10개의 분류 클래스에 따라 분류하는 모델 개발

### **데이터 설명**
- 학습 데이터 : 해양 침적 쓰레기 이미지(.jpg) 7187 장
- 라벨 데이터 : 이미지 파일명, 이미지 속 쓰레기 종류 정보가 표 형태로 담긴 데이터 (.csv) 1장

### **입출력**
- Input : 해양 쓰레기가 존재하는 수중 촬영 이미지
- Output : 쓰레기 (10종) -'bundle of ropes', 'circular fish trap', 'eel fish trap', 'rope', 'fish net', 'other objects', 'rectangular fish trap', 'tire', 'spring fish trap', 'wood' 중 하나

### **데이터셋 구성**
- Train
    - `images/` : 7187장의 jpg 파일
    - `train.csv`  : 이미지 파일명과 쓰레기 유형 정보가 표 형태로 담긴 csv 파일 1개
- test
    - `images/` : 2376장의 jpg 파일


### 베이스라인 사용 모델
- [EfficientNet B0](https://pytorch.org/vision/master/_modules/torchvision/models/efficientnet.html)
- 이 외에도 이미지 분류에 자주 사용되는 모델에는 CNN, ResNet, EfficientNet 등이 있습니다.


## 코드 구조
이 베이스라인 코드는 간단하게 아래 네 단계로 이루어져 있습니다.
- `1.세팅` : 필요한 라이브러리 설치, 학습 로그 생성을 위한 전반적인 세팅
- `2.데이터`: 사용할 데이터셋을 가져오고 모델에 전달할 Dataloader 생성
  - `class CustomDataset`: 데이터를 불러오고 (필요할 경우) 데이터 전처리 진행, 및 torch.utils.data.DataLoader의 첫번째 인자 형식으로 변환
  - `torch.utils.data.DataLoader(dataset, batch_size=, ...)`: 모델에 공급할 데이터 로더 생성
- `3.모델 설계`: 학습 및 추론에 사용할 모델 구조 설계
  - `class TrashClassifier`: 모델 구조 설계
- `4.학습`: 설계된 모델로 데이터 학습
  - 학습된 가중치 파일은 `.ipynb` 코드와 같은 경로에 .pth 형태로 저장됨
- `5.추론`: 학습된 모델을 사용해 테스트 데이터로 추론
  - 학습된 모델로 테스트 데이터에 대한 추론을 진행
  - 추론 결과는 본 `.ipynb` 코드와 같은 경로에 .csv 형태로 저장됨. 이를 플랫폼에 업로드해 점수 확인

## 세팅
### 라이브러리
- 코드 전반에 사용되는 라이브러리를 설치 및 로드합니다.

In [ ]:
# 설치되지 않은 라이브러리의 경우, 주석 해제 후 코드를 실행하여 설치
# !pip install torch
# !pip install pandas
# !pip install tqdm
# !pip install matplotlib
# !pip install opencv-python
# !pip install sklearn
# !pip install pillow

In [ ]:
# 필요한 라이브러리 불러오기
import os
import random
from tqdm import tqdm
import torch.nn as nn
from datetime import datetime, timezone, timedelta
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
import torch
import pandas as pd



### 기타
- SEED 고정 : 시드를 고정하여 실행하면 같은 코드를 여러 번 실행한 결과에 일관성을 부여합니다.
- device 설정 : GPU를 사용하기 위해서 지정합니다.
- 디렉토리 설정 : 추후 반복적으로 사용하게 될 현재 디렉토리 경로를 설정하세요.
  데이터는 현재 디렉토리의 `data/`폴더 안에 저장하세요.  
- 셋팅할 working directory 구조  
  |--코드.ipynb  
  |--data/  
  |--|--train/
  |--|--|--images/
  |--|--|--train.csv  
  |--|--test/  
  |--|--| 

In [ ]:
# SEED
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# Set device
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU 사용 가능 여부 : ", torch.cuda.is_available())

# DEBUG True 설정 시, Performance recorder가 실행되지 않습니다.
DEBUG = False

# 경로 설정

### PROJECT_DIR : 코드를 실행하며 생성될 파일들이 저장될 경로 설정 ###
PROJECT_DIR = ''
ROOT_PROJECT_DIR = os.path.dirname(PROJECT_DIR)
DATA_DIR = 'data/'

print("project directory 경로 전재 여부 : ", os.path.isdir(PROJECT_DIR))
print("데이터 경로가 옳은지 확인 : ", os.path.isdir(DATA_DIR))



### 학습 로그 기록을 위한 Performance recorder 세팅
Performance recorder는 학습 결과를 분석하는 데에 유용한 여러 정보를 저장합니다. 해당 베이스라인에서는 학습을 시작한 시점에 따라 time stamp를 생성, 그에 따라 폴더를 생성하여 Epoch 별로 학습 진행사황 등을 기록합니다.

In [ ]:

MODEL = 'TrashClassifier'

# 학습 시간 (리얼 타임)에 따라 serial 생성
KST = timezone(timedelta(hours=9))
TRAIN_TIMESTAMP = datetime.now(tz=KST).strftime("%Y%m%d%H%M%S")
TRAIN_SERIAL = f'{MODEL}_{TRAIN_TIMESTAMP}' if DEBUG is not True else ''

# 학습 performance 기록 경로 설정
PERFORMANCE_RECORD_DIR = os.path.join('results', 'train', TRAIN_SERIAL)

PERFORMANCE_RECORD_COLUMN_NAME_LIST = ['train_serial', 'train_timestamp', 'model_str', 'optimizer_str', 
                                       'loss_function_str', 'metric_function_str', 'early_stopping_patience', 
                                       'batch_size', 'epoch', 'learning_rate', 'momentum', 'random_seed', 'epoch_index', 
                                       'train_loss', 'validation_loss', 'train_score', 'validation_score', 'elapsed_time']


In [ ]:
# 학습 결과 저장할 폴더 만들기
def make_directory(directory: str) -> str:
    """경로가 없으면 생성
    Args:
        directory (str): 새로 만들 경로

    Returns:
        str: 상태 메시지
    """

    try:
        if not os.path.isdir(directory):
            os.makedirs(directory)
            msg = f"Directory created {directory}"

        else:
            msg = f"{directory} already exists"

    except OSError as e:
        msg = f"Fail to create directory {directory} {e}"

    return msg

make_directory(PERFORMANCE_RECORD_DIR)

In [ ]:
import logging

# Set system logger
def get_logger(name: str, file_path: str, stream=False) -> logging.RootLogger:
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter('%(asctime)s | %(name)s | %(levelname)s | %(message)s')
    stream_handler = logging.StreamHandler()
    file_handler = logging.FileHandler(file_path)

    stream_handler.setFormatter(formatter)
    file_handler.setFormatter(formatter)

    if stream:
        logger.addHandler(stream_handler)
    logger.addHandler(file_handler)

    return logger

system_logger = get_logger(name='train', file_path=os.path.join(PERFORMANCE_RECORD_DIR, 'train_log.log'))

## EDA (Explaratory Data Analylsis) - 데이터 확인
pandas 라이브러리를 이용해 데이터를 간단하게 살펴보겠습니다.  
데이터를 이해하기 위해 더 필요하다고 생각되는 부분을 각자 추가해보세요.

In [ ]:
img = os.listdir(os.path.join(DATA_DIR, 'train', 'images'))
print("학습할 이미지 갯수 : ", len(img))


- 이미지 샘플 하나 확인


In [ ]:
import matplotlib.pyplot as plt
from cv2 import cv2

sampleImage = os.path.join(DATA_DIR, 'train', 'images',img[4])


def getImg(path):
    bgr_image = cv2.imread(path)
    rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
    return rgb_image

sampleImage = getImg(sampleImage)

plt.imshow(sampleImage)

- train.csv 파일 확인

In [ ]:
train_csv = pd.read_csv(os.path.join(DATA_DIR, 'train', "train.csv"))
train_csv.sample(n=10)

- 학습 데이터의 쓰레기 분류 분포

In [ ]:
train_csv.groupby('category').count()

- 카테고리 딕셔너리 생성

In [ ]:
category_list = train_csv['category'].unique()
category = {}
for i in range(len(category_list)):
    category[category_list[i]] = i
print(category)

In [ ]:
category.keys()

### 데이터 전처리 방법
해당 과제는 이미 전처리(resize 등)가 적용된 이미지 데이터를 사용합니다. 해당 베이스라인에서 직접 다루지는 않지만, 이미지 분류 과제에서 자주 사용되는 resize, cubic interpolation 등을 짚고 넘어갑시다.
- resize : 이미지 파일의 크기가 너무 커서 학습 시간이 오래걸리거나, 각 이미지 파일의 크기가 다른 점을 보정하기 위해 지정된 규격으로 이미지 크기를 통일합니다. 
- interpolation : 이미지의 비율을 변경할 때 존재하지 않는 영역에 새로운 픽셀 값을 매핑하거나, 존재하는 픽셀들을 압축하여 새로운 값을 할당해야 합니다. 새로운 픽셀 값은 interpolation(보간법)을 이용하여 구합니다. 여러 보간법이 존재하며, 상황에 따라 다른 보간법을 적용하는 것이 보편적입니다. 
 

## 데이터 로드
### 사용할 파라미터
- `BATCH_SIZE` : 모델에 입력할 데이터의 단위입니다. 전체 데이터 셋을 여러 개의 소그룹으로 나누어 학습하게 되는데,  batch size는 하나의 소그룹에 속하는 데이터 수가 됩니다 (전체 1000개의 데이터에서 배치 사이즈가 50 이라면 학습 시 1 epoch당 20 iteration을 돌며 전체 데이터를 한번 학습 하게 됩니다).


In [ ]:
# 데이터로드 파라미터
BATCH_SIZE = 128
INPUT_SHAPE = (128, 128)

# 전체 학습 데이터에서 학습(train), 검증(validation) 셋으로 나눌 때, 검증 셋 비율을 설정합니다.
VAL_RATIO = 0.2

In [ ]:
import os
import copy
from cv2 import cv2
import torch
import sys
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split



class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):

        # !!!!! 데이터 경로 확인 !!!!!
        self.data_dir = os.path.join(data_dir,'train')
        self.mode = mode
        self.input_shape = input_shape
        self.db = self.data_loader()
        self.transform = transforms.Compose([transforms.Resize(self.input_shape), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.class_num = len(self.db['label'].unique())
        
    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        
        img_dir = os.path.join(self.data_dir, 'images')
        label_dir = os.path.join(self.data_dir, "train.csv")
        
        # 이미지 폴더 존재 여부 체크
        if not os.path.isdir(img_dir):
            print(f'!!! Cannot find {img_dir}... !!!')
            sys.exit()
            
        # 라벨 파일 존재 여부 체크
        if not os.path.lexists(label_dir):
            print(f'!!! Cannot find {label_dir}... !!!')
            sys.exit()
        
        category = {'bundle of ropes': 0, 'circular fish trap': 1, 'eel fish trap': 2, 
                     'rope': 3, 'fish net': 4, 'other objects': 5, 'rectangular fish trap': 6, 
                     'tire': 7, 'spring fish trap': 8, 'wood': 9}

        category_ls = list(category.keys())
        
        # train.csv를 pandas로 불러오기
        df = pd.read_csv(label_dir)
        
        # return할 train, val 데이터 초기화 - pandas dataframe 형태로 저장할 예정
        db_train = db_val = pd.DataFrame(columns=['img_path','label'])

        # 카테고리별로 train/val 비율 맞춰 나누기 위해 카테고리별로 루프 실행
        for cat in category_ls:

            img_path_list = []
            img_label_list = []
            
            # 특정 카테고리에 해당하는 이미지 리스트 뽑기
            img_list = df["file_name"].loc[df['category']==cat].values

            for filename in img_list:
                if os.path.lexists(os.path.join(img_dir, filename)):
                    img_path_list.append(os.path.join(img_dir, filename))
                    img_label_list.append(category[cat])

                else:
                    print("cannot find ", os.path.join(img_dir,filename))

                    
            db = pd.DataFrame({'img_path': img_path_list, 'label': img_label_list}) 
            
            # 학습, 검증 셋 나누기
            train, val = train_test_split(db, test_size=VAL_RATIO, random_state=42, shuffle=True)
            
            db_train = pd.concat([db_train,train])
            db_val = pd.concat([db_val,val])
 
        db_train = db_train.sample(frac=1).reset_index(drop=True)
        db_val = db_val.sample(frac=1).reset_index(drop=True)

        if self.mode=='train':
            db = db_train

        elif self.mode=='val':
            db = db_val
        else:
            print("Please check your mode : ", mode, " must be either train or val")
       
        print(db.head())
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # 1. load image
        cvimg = cv2.imread(data['img_path'], cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['img_path'])

        # 2. preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        return trans_image, data['label']
    
    
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

print('\nTrain set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

## 모델 설계 / pretrained model 불러오기
### 사용할 파라미터
- `LEARNING_RATE` : 
  - 경사하강법(Gradient Descent)을 통해 loss function의 minimum값을 찾아다닐 때, 그 탐색 과정에 있어서의 보폭 정도로 직관적으로 이해 할 수 있습니다. 보폭이 너무 크다면 최적값을 쉽게 지나칠 위험이 있고, 보폭이 너무 작다면 탐색에 걸리는 시간이 길어집니다.
- `EPOCHS` : 
  - 한 번의 epoch는 인공 신경망에서 전체 데이터 셋에 대해 forward pass/backward pass 과정을 거친 것입니다.
  - 즉, epoch이 1만큼 지나면, 전체 데이터 셋에 대해 한번의 학습이 완료된 상태입니다.
  - 모델을 만들 때 적절한 epoch 값을 설정해야만 underfitting과 overfitting을 방지할 수 있습니다.
  - 1 epoch 에 (데이터 갯수 / batch size) interations 실행
- `EARLY_STOPPING_PATIENCE` :
  - 너무 많은 epoch은 overfitting을 일으키고, 너무 적은 epoch은 underfitting을 일으킵니다. 이런 딜레마에 빠지지 않기 위도록 특정 시점에 학습을 멈추는 방법이 early stopping입니다.
  - 해당 변수는 validation score가 개선되지 않아도 학습을 몇 에폭 더 진행할 지 결정합니다. 예를 들어 EARLY_STOPPING_PATIENCE를 5로 설정하고 validation score가 10 에폭에서 가장 높았지만 다음 에폭부터 줄어든다면, 15에폭까지는 학습을 진행하며 validation score가 더 높아지는지 확인하고, 그렇지 않다면 학습을 중단합니다.
- `WEIGHT_DECAY` : 
  - overfitting을 억제하는 학습 기법의 하나로, 학습된 모델의 복잡도를 줄이기 위해서 학습 중 weight가 너무 큰 값을 가지지 않도록 Loss function에 Weight가 커질 경우에 대한 패널티 항목을 넣습니다.

In [ ]:
# hyper-parameters
LEARNING_RATE = 0.0005
EPOCHS = 10

EARLY_STOPPING_PATIENCE = 10
WEIGHT_DECAY = 0.00001

OPTIMIZER = ''
SCHEDULER = ''
MOMENTUM = ''
LOSS_FN = ''
METRIC_FN = ''

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from efficientnet_pytorch import EfficientNet

class TrashClassifier(nn.Module):
    def __init__(self, num_class):
        super(TrashClassifier, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_class)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x
    
# 모델 생성하기
model = TrashClassifier(num_class=train_dataset.class_num).to(device)

### 학습에 필요한 함수 설정
- `optimizer` : 
  - 손실함수 값이 최소가 되는 부분을 찾기 위해 학습율과 기울기를 다양하게 수정하여 가중치를 변경시키는 것을 최적화라고 하고, 최적화의 다양한 방식들을 옵티마이저라고 합니다.
- `scheduler` : 
  - Learning rate scheduler는 미리 학습 일정을 정해두고, 그 일정에 따라 학습률을 조정하게 합니다. 다시 말하면 Learning rate가 어떻게 변화하게 할 지 정합니다.
- `metric_fn` :
  - 학습에서 평가 지표(metric)는 validation에서 훈련된 모델이 얼마나 잘 학습되고 있는지 확인하며, 훈련 과정을 모니터링 하는데 사용합니다. 과제와 학습 모델에 따라 다양한 평가 지표가 사용됩니다.
  - 해당 베이스라인에서는 macro f1 score를 평가 지표로 사용합니다.

In [ ]:
import sklearn.metrics as metrics

def get_metric_fn(y_pred, y_answer, y_prob):
    """ Metric 함수 반환하는 함수

    Returns:
        metric_fn (Callable)
    """
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    f1 = metrics.f1_score(y_answer, y_pred,average="macro")
    return f1, 1

# Set optimizer, scheduler, loss function, metric function, criterion (loss function)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
criterion = nn.CrossEntropyLoss()
metric_fn = get_metric_fn

### Trainer 설정
- epoch별 학습/검증 절차를 정의하는 Trainer class 입니다.

In [ ]:
import tqdm

class Trainer():
    """ Trainer
        epoch에 대한 학습 및 검증 절차 정의
    """

    def __init__(self, criterion, model, device, metric_fn, optimizer=None, scheduler=None, logger=None):
        """ 초기화
        """
        self.criterion = criterion
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.logger = logger
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []
        for batch_index, (img, label) in enumerate(dataloader):
            
            img = img.to(self.device)
            label = label.to(self.device).long()
            pred = self.model(img)
            loss = self.criterion(pred, label)
            
            ## Pytorch에서는 gradients값들을 추후에 backward를 해줄때 계속 더해주기 때문에 
            ## 항상 backpropagation을 하기전에 gradients를 zero로 만들어주고 시작을 해야합니다.
            self.optimizer.zero_grad()
            
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, auroc = self.metric_fn(y_pred=pred_lst, y_answer=target_lst, y_prob=prob_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}'
        print(msg)
        #self.logger.info(msg) if self.logger else print(msg)

    def validate_epoch(self, dataloader, epoch_index, mode=None):
        """ 한 epoch에서 수행되는 검증 절차

        Args:
            dataloader (`dataloader`)
            epoch_index (int)
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []
        with torch.no_grad():
            for batch_index, (img, label) in enumerate(dataloader):
                img = img.to(self.device)
                label = label.to(self.device).long()
                pred = self.model(img)
                loss = self.criterion(pred, label)
                val_total_loss += loss.item()
                prob_lst.extend(pred[:, 1].cpu().tolist())
                target_lst.extend(label.cpu().tolist())
                pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            self.val_mean_loss = val_total_loss / batch_index
            self.validation_score, auroc = self.metric_fn(y_pred=pred_lst, y_answer=target_lst, y_prob=prob_lst)
            msg = f'Epoch {epoch_index}, {mode} loss: {self.val_mean_loss}, Acc: {self.validation_score}, ROC: {auroc}'
            print(msg)
        #self.logger.info(msg) if self.logger else print(msg)

        
# Set trainer
trainer = Trainer(criterion, model, device, metric_fn, optimizer, scheduler, logger=system_logger)

### Earlystopper 세팅

In [ ]:
import numpy as np
import logging

class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int, verbose: bool, logger:logging.RootLogger=None)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.verbose = verbose

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.logger = logger
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단

        Args:
            loss (float):

        Examples:
            
        Note:
            
        """  

        if self.min_loss == np.Inf:
            #첫 에폭
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

        elif loss > self.min_loss:
            # loss가 줄지 않음 -> patience_counter 1 증가
            self.patience_counter += 1
            msg = f"Early stopper, Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)
                
        elif loss <= self.min_loss:
            # loss가 줄어듬 -> min_loss 갱신
            self.save_model = True
            msg = f"Early stopper, Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
            # self.save_checkpoint(loss=loss, model=model)

            if self.verbose:
                self.logger.info(msg) if self.logger else print(msg)
                
# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE, verbose=True, logger=system_logger)


### Performance Recorder 세팅

In [ ]:
import os
import csv
import numpy as np
import logging
from matplotlib import pyplot as plt

import torch


class PerformanceRecorder():

    def __init__(self,
                 column_name_list: list,
                 record_dir: str,
                 key_column_value_list: list,
                 model: 'model',
                 optimizer: 'optimizer',
                 scheduler: 'scheduler',
                 logger: logging.RootLogger=None):
        """Recorder 초기화
            
        Args:
            column_name_list (list(str)):
            record_dir (str):
            key_column_value_list (list)

        Note:
        """

        self.column_name_list = column_name_list
        
        self.record_dir = record_dir
        self.record_filepath = os.path.join(self.record_dir, 'record.csv')
        self.best_record_filepath = os.path.join('/'.join(self.record_dir.split('/')[:-1]),'train_best_record.csv')
        self.weight_path = os.path.join(record_dir, 'model.pt')

        self.row_counter = 0
        self.key_column_value_list = key_column_value_list

        self.train_loss_list = list()
        self.validation_loss_list= list()
        self.train_score_list = list()
        self.validation_score_list = list()

        self.loss_plot = None
        self.score_plot = None

        self.min_loss = np.Inf
        self.best_record = None

        self.logger = logger
        self.model = model
        self.optimizer = optimizer
        self.scheduler = scheduler

        self.key_column_value_list = key_column_value_list

    def set_model(self, model: 'model'):
        self.model = model

    def set_logger(self, logger: logging.RootLogger):
        self.logger = logger

    def create_record_directory(self):
        """
        record 경로 생성
        """
        make_directory(self.record_dir)
        msg = f"Create directory {self.record_dir}"
        self.logger.info(msg) if self.logger else None

    def add_row(self,
                epoch_index: int,
                train_loss: float,
                validation_loss: float,
                train_score: float,
                validation_score: float):
        """Epoch 단위 성능 적재
        
        최고 성능 Epoch 모니터링
        모든 Epoch 종료 이후 최고 성능은 train_best_records.csv 에 적재

        Args:
            row (list): 

        """
        self.train_loss_list.append(train_loss)
        self.validation_loss_list.append(validation_loss)
        self.train_score_list.append(train_score)
        self.validation_score_list.append(validation_score)

        row = self.key_column_value_list + [epoch_index, train_loss, validation_loss, train_score, validation_score]
        
        # Update scheduler learning rate
        learning_rate = self.scheduler.get_last_lr()[0]
        row[9] = learning_rate

        with open(self.record_filepath, newline='', mode='a') as f:
            writer = csv.writer(f)

            if self.row_counter == 0:
                writer.writerow(self.column_name_list)

            writer.writerow(row)
            msg = f"Write row {self.row_counter}"
            self.logger.info(msg) if self.logger else None

        self.row_counter += 1

        # Update best record & Save check point
        if validation_loss < self.min_loss:
            msg = f"Update best record row {self.row_counter}, checkpoints {self.min_loss} -> {validation_loss}"
            
            self.min_loss = validation_loss
            self.best_record = row
            self.save_weight()

            self.logger.info(msg) if self.logger else None

    def add_best_row(self):
        """
        모든 Epoch 종료 이후 최고 성능에 해당하는 row을 train_best_records.csv 에 적재
        """

        n_row = count_csv_row(self.best_record_filepath)

        with open(self.best_record_filepath, newline='', mode='a') as f:
            writer = csv.writer(f)

            if n_row == 0:
               writer.writerow(self.column_name_list)
            
            writer.writerow(self.best_record)

        msg = f"Save best record {self.best_record_filepath}"
        self.logger.info(msg) if self.logger else None

    def save_weight(self)-> None:
        """Weight 저장

        Args:
            loss (float): validation loss
            model (`model`): model
        
        """
        check_point = {
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict()
        }
        torch.save(check_point, self.weight_path)
        #torch.save(self.model.state_dict(), self.weight_path)
        msg = f"Model saved: {self.weight_path}"
        self.logger.info(msg) if self.logger else None

    def save_performance_plot(self, final_epoch: int):
        """Epoch 단위 loss, score plot 생성 후 저장

        """
        self.loss_plot = self.plot_performance(epoch=final_epoch+1,
                                          train_history=self.train_loss_list,
                                          validation_history=self.validation_loss_list,
                                          target='loss')
        self.score_plot = self.plot_performance(epoch=final_epoch+1,
                                           train_history=self.train_score_list,
                                           validation_history=self.validation_score_list,
                                           target='score')

        self.loss_plot.savefig(os.path.join(self.record_dir, 'loss.png'))
        self.score_plot.savefig(os.path.join(self.record_dir, 'score.jpg'))
        plt.close('all')


        msg = f"Save performance plot {self.record_dir}"
        self.logger.info(msg) if self.logger else None

    def plot_performance(self,
                         epoch: int,
                         train_history:list,
                         validation_history:list,
                         target:str)-> plt.figure:
        """loss, score plot 생성

        """
        fig = plt.figure(figsize=(12, 5))
        epoch_range = list(range(epoch))

        plt.plot(epoch_range, train_history, marker='.', c='red', label="train")
        plt.plot(epoch_range, validation_history, marker='.', c='blue', label="validation")

        plt.legend(loc='upper right')
        plt.grid()
        plt.xlabel('epoch')
        plt.ylabel(target)

        return fig

In [ ]:
key_column_value_list = [
    TRAIN_SERIAL,
    TRAIN_TIMESTAMP,
    MODEL,
    OPTIMIZER,
    LOSS_FN,
    METRIC_FN,
    EARLY_STOPPING_PATIENCE,
    BATCH_SIZE,
    EPOCHS,
    LEARNING_RATE,
    WEIGHT_DECAY,
    RANDOM_SEED]

performance_recorder = PerformanceRecorder(column_name_list=PERFORMANCE_RECORD_COLUMN_NAME_LIST,
                                           record_dir=PERFORMANCE_RECORD_DIR,
                                           key_column_value_list=key_column_value_list,
                                           logger=system_logger,
                                           model=model,
                                           optimizer=optimizer,
                                           scheduler=scheduler)


### 학습
- 각 에폭별 수행할 절차를 trainer 클래스의 train_epoch, validation_epoch을 이용해 정의합니다.  
- EPOCHS만큼, 혹은 early_stop이 될 때까지 에폭을 반복합니다.

In [ ]:
from tqdm import tqdm

criterion = 1E+8

# 저장될 모델 이름
model_name = 'best.pt'

# 에폭 별로 train_epoch, valid_epoch 실헹힘
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index, 'val')

    # Performance record - csv & save elapsed_time
    performance_recorder.add_row(epoch_index=epoch_index,
                                 train_loss=trainer.train_mean_loss,
                                 validation_loss=trainer.val_mean_loss,
                                 train_score=trainer.train_score,
                                 validation_score=trainer.validation_score)

    # Performance record - plot
    performance_recorder.save_performance_plot(final_epoch=epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break
    
    if trainer.val_mean_loss < criterion:
        criterion = trainer.val_mean_loss
        performance_recorder.weight_path = os.path.join(PERFORMANCE_RECORD_DIR, model_name)
        performance_recorder.save_weight()
        # print(f'{epoch_index} model saved train acc :{trainer.train_score}, val acc :{trainer.validation_score}')
        print(f'{epoch_index} model saved')
        print('----------------------------------')


print("Model saved: ", os.path.join(PERFORMANCE_RECORD_DIR, model_name))
print("Train score saved: ",os.path.join(PERFORMANCE_RECORD_DIR, 'score.jpg'))
print("Train loss saved: ",os.path.join(PERFORMANCE_RECORD_DIR, 'loss.png'))

## 추론 (Predict)
테스트 데이터의 타겟 변수를 `sample_submission.csv` 양식에 맞춰 저장한 파일을 인공지능 놀이터 플랫폼을 통해 제출하면 추론 점수를 확인할 수 있습니다.

"category" 컬럼 값을 여러분의 모델의 추론 결과로 채워 제출 파일을 만듭니다 (현재는 모두 아래 보시는 바와 같이 동일한 값으로 채워져 있습니다). "file_name" 값을 기준으로 채점을 진행하는 점 유의해주시기 바랍니다.

### 제출 파일 양식 확인

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
sample_submission.head()

### 추론을 위한 라이브러리 불러오기 및 세팅 

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import torch
import cv2
import sys
import copy
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch

# Set device
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())


# # CONFIG
PROJECT_DIR = ''
ROOT_PROJECT_DIR = os.path.dirname(PROJECT_DIR)
DATA_DIR = 'data/'

# SEED
RANDOM_SEED = 42


In [ ]:
# train 데이터로부터 카테고리 갯수 가져와 지정 

train_csv = pd.read_csv(os.path.join(DATA_DIR, 'train', "train.csv"))

# 카테고리 갯수 = (분류) class 갯수 
num_category = len(train_csv['category'].unique())

In [ ]:
# PREDICT
BATCH_SIZE = 16
INPUT_SHAPE = (128, 128)
NUM_CLASS = num_category


In [ ]:
# 시드 고정
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# 평가 지표 함수, criterion (loss function) 설정
import sklearn.metrics as metrics

def get_metric_fn(y_pred, y_answer, y_prob):
    """ Metric 함수 반환하는 함수

    Returns:
        metric_fn (Callable)
    """
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    f1 = metrics.f1_score(y_answer, y_pred,average="macro")
    return f1, 1

metric_fn = get_metric_fn
criterion = nn.CrossEntropyLoss()

### 테스트 데이터 로드

In [ ]:
# TestDataset 만드는 클래스

class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = os.path.join(data_dir,'test')
        self.input_shape = input_shape
        self.db = self.data_loader()
        self.transform = transforms.Compose([transforms.Resize(self.input_shape), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        root = os.path.join(self.data_dir, 'images')
        
        if os.path.isdir(root):
            img_list = os.listdir(root)
        else:
            print(root, " doesn't exist")
        img_path_list = []
        
        for filename in img_list:
            ext = os.path.splitext(filename)[-1]
            if (os.path.lexists(os.path.join(root, filename))):
                if '.jpg' == ext:
                     img_path_list.append(os.path.join(root, filename))
            else:
                print("please check your test set directory .... ")
                break

        db = pd.DataFrame({'img_path': img_path_list})
        
        print(db.sample(n=5))
        print("Train set samples: ", len(db))
        
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
         #1. load image
        cvimg = cv2.imread(data['img_path'], cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['img_path'])

        # 2. preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        return trans_image, data['img_path'].split('/')[-1]


In [ ]:
# 테스트 데이터 로드   
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 학습한 가중치 파일 불러오기

In [ ]:
# pre-trained model 함수
from efficientnet_pytorch import EfficientNet

class TrashClassifier(nn.Module):
    def __init__(self, num_class):
        super(TrashClassifier, self).__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=num_class)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x

In [ ]:
# 앞서 모델이 저장된 경로를 입력하세요
# 예시 :  'results/train/TrashClassifier_202111025172155/best.pt'
TRAINED_MODEL_PATH = ###

print("학습한 모델 weight 파일이 경로에 존재하나요? - ", os.path.lexists(TRAINED_MODEL_PATH))
model = TrashClassifier(num_class=NUM_CLASS).to(device)
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])


### Predict ! (test trainer)
- 분류 카테고리에 0부터 9까지의 정수 코드를 부여해 학습하였으므로 추론된 output도 정수일 것입니다. 이를 다시 단어로 바꿔주는 디코딩 작업도 잊지 마세요.

In [ ]:
# 제출할 submission.csv 파일명
SAVE_PATH = 'submission.csv'

# 디코딩을 위한 딕셔너리
category = {'bundle of ropes': 0, 'circular fish trap': 1, 'eel fish trap': 2, 
             'rope': 3, 'fish net': 4, 'other objects': 5, 'rectangular fish trap': 6, 
             'tire': 7, 'spring fish trap': 8, 'wood': 9}
decode = dict(map(reversed, category.items()))


file_name_lst = []
pred_lst = []

with torch.no_grad():
    for batch_index, (img, file_name) in enumerate(test_dataloader):
        img = img.to(device)
        pred = model(img)
        pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        file_name_lst.extend(file_name)
print("Prediction completed")        

# 디코딩
pred_lst = list(map(lambda x: decode[x], pred_lst))
print("decoding completed")

### 결과 제출 파일 확인

In [ ]:
pred = pd.DataFrame({'file_name':file_name_lst, 'category':pred_lst})
pred.head()

### 결과 제출 파일 만들기

In [ ]:
pred.to_csv(SAVE_PATH, index=False)
print(SAVE_PATH, "saved.")

-> 이 파일을 홈페이지에 업로드해 점수를 확인해보세요!